In [1]:
#!pip install -U tensorflow
#!pip install -U keras

import tensorflow as tf
import keras
from keras.models import load_model
import numpy as np
import h5py
import os
import pandas as pd
from pandas import DataFrame as df

from ibm_botocore.client import Config
import ibm_boto3
from IPython.display import HTML
from keras.models import model_from_json

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3


Using TensorFlow backend.


# Preparing The Data
In the below cells, we load the data from IBM Cloud into our working notebook directory.
We also demonstrate IBM Watson Studio's integration with .csv files and how they can load into a pandas DataFrame seamlessly.

In [2]:
def download_file_cos(credentials,local_file_name,key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
        

In [3]:
#For this cell, First click on this cell and make sure it is highlighted.
# Then click on the data icon (the 1s and 0s) on the right side of the screen
#and navigate to eval.h5 and click on "Insert to code" and "Insert Credentials" 
#It should produce a code snippet similar to this:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_eval = {
    'IAM_SERVICE_ID': 'xxxx',
    'IBM_API_KEY_ID': 'xxxx',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'xxxx',
    'FILE': 'eval.h5'
}

# Note: the xxxx here is to indicate that that info will be populated based on your access token keys by Watson Studio
# Make sure you name the dictionary `credentials_eval`

In [4]:
#For this cell, First click on this cell and make sure it is highlighted.
# Then click on the data icon (the 1s and 0s) on the right side of the screen
#and navigate to classifier_model.h5 or model_final.h5 and click on "Insert to code" and "Insert Credentials" 
#It should produce a code snippet similar to this:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_model = {
    'IAM_SERVICE_ID': 'xxxx',
    'IBM_API_KEY_ID': 'xxxx',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'xxxx',
    'FILE': 'classifier_model.h5' #(or alternately `model_final.h5`)
}

# Note: the xxxx here is to indicate that that info will be populated based on your access token keys by Watson Studio
# Make sure you name the dictionary `credentials_model`

In [5]:
#Downloading the files from IBM Cloud to the current directory
#Downloading the model weights as `final_weights.h5` regardless of original name.
download_file_cos(credentials_model,'final_weights.h5','classifier_model.h5')
download_file_cos(credentials_eval,'eval.h5','eval.h5')

File Downloaded
File Downloaded


In [6]:
#Repeating the above steps, we will not load the csv files into Pandas DataFrame object directly.
#Navigate to the class_labels_indices.csv file on the right and click on "Insert to code" -> "Insert pandas dataframe"
#It should produce a code snippet similar to below.

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_b814de19900a4db5acdb554945aa9517 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xxxxx',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_b814de19900a4db5acdb554945aa9517.get_object(Bucket='audiosetclassification-donotdelete-pr-65xlshluvaajgp',Key='class_label_indices.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

#Note: the "xxxx" would be replaced with your API key when you click on "Insert to code" -> "Insert pandas dataframe".

#Once inserted, delete / comment out the above demo code. 


,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation


In [7]:
#Repeat the same for "eval_segments.csv" file

body = client_b814de19900a4db5acdb554945aa9517.get_object(Bucket='audiosetclassification-donotdelete-pr-65xlshluvaajgp',Key='eval_segments.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

    
# This time we have some pre-processing steps as we do not want to import all columns and rows
# Make sure you pass the following arguments with `pd.read_csv()`
df_data_2 = pd.read_csv(body, header=None, names=['video_id', 'start_time', 'end_time'], skiprows=3, usecols=[0,1,2]) #'# YTID', ' start_seconds', ' end_seconds'])
df_data_2.head()

,video_id,start_time,end_time
0,--4gqARaEJE,0.0,10.0
1,--BfvyPmVMo,20.0,30.0
2,--U7joUcTCo,0.0,10.0
3,--i-y1v8Hy8,0.0,9.0
4,-0BIyqJj9ZU,30.0,40.0


In [8]:
#Loading the evaluation data and labels to numpy objects
def load_data(hdf5_path):
    """
    Loads the data into numpy objects. 
    Input : Path to data.
    Output : Train/Test examples, corresponding labels, corresponding youtube video_id.
    """
    with h5py.File(hdf5_path, 'r') as hf:
        x = hf.get('x')
        y = hf.get('y')
        video_id_list = hf.get('video_id_list')
        x = np.array(x)
        y = list(y)
        video_id_list = list(video_id_list)
        
    return x, y, video_id_list

def uint8_to_float32(x):
    return (np.float32(x) - 128.) / 128.
    
def bool_to_float32(y):
    return np.float32(y)

(x, y, video_id_list) = load_data('eval.h5')
x = uint8_to_float32(x)		# shape: (N, 10, 128)
y = bool_to_float32(y)		# shape: (N, 527)


# Building the Model
In the below cells we build the model and load the weights pre-trained on IBM cloud/dlaas.

In [9]:
#Building the model

import keras
from keras.models import Model
from keras.layers import (Input, Dense, BatchNormalization, Dropout, Lambda,
                          Activation, Concatenate)
import keras.backend as K
from keras.optimizers import Adam

try:
    import cPickle
except BaseException:
    import _pickle as cPickle


def average_pooling(inputs, **kwargs):
    input = inputs[0]   # (batch_size, time_steps, freq_bins)
    return K.mean(input, axis=1)


def max_pooling(inputs, **kwargs):
    input = inputs[0]   # (batch_size, time_steps, freq_bins)
    return K.max(input, axis=1)


def attention_pooling(inputs, **kwargs):
    [out, att] = inputs

    epsilon = 1e-7
    att = K.clip(att, epsilon, 1. - epsilon)
    normalized_att = att / K.sum(att, axis=1)[:, None, :]

    return K.sum(out * normalized_att, axis=1)


def pooling_shape(input_shape):

    if isinstance(input_shape, list):
        (sample_num, time_steps, freq_bins) = input_shape[0]

    else:
        (sample_num, time_steps, freq_bins) = input_shape

    return (sample_num, freq_bins)

model_type = 'decision_level_multi_attention'
time_steps = 10
freq_bins = 128
classes_num = 527

# Hyper parameters
hidden_units = 1024
drop_rate = 0.5
batch_size = 500

# Embedded layers
input_layer = Input(shape=(time_steps, freq_bins))

a1 = Dense(hidden_units)(input_layer)
a1 = BatchNormalization()(a1)
a1 = Activation('relu')(a1)
a1 = Dropout(drop_rate)(a1)

a2 = Dense(hidden_units)(a1)
a2 = BatchNormalization()(a2)
a2 = Activation('relu')(a2)
a2 = Dropout(drop_rate)(a2)

a3 = Dense(hidden_units)(a2)
a3 = BatchNormalization()(a3)
a3 = Activation('relu')(a3)
a3 = Dropout(drop_rate)(a3)

# Pooling layers
if model_type == 'decision_level_max_pooling':
    '''Global max pooling.

    [1] Choi, Keunwoo, et al. "Automatic tagging using deep convolutional 
    neural networks." arXiv preprint arXiv:1606.00298 (2016).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    output_layer = Lambda(max_pooling, output_shape=pooling_shape)([cla])

elif model_type == 'decision_level_average_pooling':
    '''Global average pooling.

    [2] Lin, Min, et al. Qiang Chen, and Shuicheng Yan. "Network in 
    network." arXiv preprint arXiv:1312.4400 (2013).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    output_layer = Lambda(
        average_pooling,
        output_shape=pooling_shape)(
        [cla])

elif model_type == 'decision_level_single_attention':
    '''Decision level single attention pooling.

    [3] Kong, Qiuqiang, et al. "Audio Set classification with attention
    model: A probabilistic perspective." arXiv preprint arXiv:1711.00927
    (2017).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    att = Dense(classes_num, activation='softmax')(a3)
    output_layer = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla, att])

elif model_type == 'decision_level_multi_attention':
    '''Decision level multi attention pooling.

    [4] Yu, Changsong, et al. "Multi-level Attention Model for Weakly
    Supervised Audio Classification." arXiv preprint arXiv:1803.02353
    (2018).
    '''
    cla1 = Dense(classes_num, activation='sigmoid')(a2)
    att1 = Dense(classes_num, activation='softmax')(a2)
    out1 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla1, att1])

    cla2 = Dense(classes_num, activation='sigmoid')(a3)
    att2 = Dense(classes_num, activation='softmax')(a3)
    out2 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla2, att2])

    b1 = Concatenate(axis=-1)([out1, out2])
    b1 = Dense(classes_num)(b1)
    output_layer = Activation('sigmoid')(b1)

elif model_type == 'feature_level_attention':
    '''Feature level attention.

    [1] To be appear.
    '''
    cla = Dense(hidden_units, activation='linear')(a3)
    att = Dense(hidden_units, activation='sigmoid')(a3)
    b1 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla, att])

    b1 = BatchNormalization()(b1)
    b1 = Activation(activation='relu')(b1)
    b1 = Dropout(drop_rate)(b1)

    output_layer = Dense(classes_num, activation='sigmoid')(b1)

else:
    raise Exception("Incorrect model_type!")

# Build model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 128)      0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1024)     132096      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 10, 1024)     4096        dense_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activat

In [10]:
#Loading weights
model.load_weights("final_weights.h5")

## Demo supporting code 
This is the supporting code to run the demo at the end of this notebook.

In [11]:
def demo(v):
    """
    A function to demonstrate the audio classification.
    Input : a random number to retrieve a query video.
    Output : video embed string for the YouTube video.
    Prints : Top 5 class probabilities from the classifier.
    """
    test_data = x[v:v+1,:,:]
    current_infer = model.predict(test_data)
    current_video = str(video_id_list[v],'utf-8')
    print(current_video)
    start_time = int(df_data_2.loc[df_data_2['video_id'] == current_video]['start_time'])
    video_string = '<iframe width="560" height="315" src="https://www.youtube.com/embed/'+current_video+'?autoplay=1&start='+str(start_time)+';autoplay=1 frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>'
    print('Predicted Labels :\n')
    print(df_data_1.loc[current_infer[0].argsort()[-5:][::-1]]['display_name'])
    #print('Ground Truth Labels :\n')
    #print(df_data_1.loc[y[v].argsort()[-5:][::-1]]['display_name'])
    print("Playing video snippet below.. Turn on the speakers..\n")
    return video_string

# Real Time Demo

Replace the number below to change the video and generate a real-time inference.

In [12]:
#Try out a random number between 0 and 20000 
video_number = 800
HTML(demo(video_number))

10YXuv9Go0E
Predicted Labels :

342                   Skateboard
137                        Music
0                         Speech
509    Outside, urban or manmade
468                Whack, thwack
Name: display_name, dtype: object
Playing video snippet below.. Turn on the speakers..



/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# Reverse Search Audio using keywords

In this section we will first perform inference on the full test dataset and store it to later retrieve search results.

In [13]:
#First we get the inference for all our data and store it under `inferences`
inferences = model.predict(x, batch_size=100)
#If you are facing memory/time issues, try running for a smaller subset
#inferences = model.predict(x[0:1000])

In [14]:
#Now we write a function to retrieve the required audios
def search_audio(keyword, infer=inferences):
    """
    Uses the inferences made using the audioset classifier to lookup and retrieve relevant audio/video for a given keyword.
    Inputs : search query, inferences on test dataset.
    Outputs : YouTube embed link for the strongest video returned and a list of all videos containing the search query.
    """
    try:
        class_id = int(df_data_1.loc[df_data_1['display_name'] == keyword]['index'])
        print("Retrieving videos containing "+keyword +" with the highest probabilities...")
    except:
        class_id = -1
        print("Search Query is invalid!")
    strong_inferences = np.argsort(inferences,axis = 1)[:,-5:]
    video_ids = np.where(strong_inferences==class_id)[0]
    video_strings = []
    max_prob = -1
    strongest_video_embed = ''
    print_counter = 0
    if(video_ids.any()):
        print("...Done!")
        print("Printing a few strong results.. ")
    for v in video_ids:
        current_video = str(video_id_list[v],'utf-8')
        start_time = df_data_2.loc[df_data_2['video_id'] == current_video]['start_time'].astype(int)
        video_string = 'https://www.youtube.com/watch?v='+current_video+'&t='+str(start_time)
        if(inferences[v][class_id]>0.8 and print_counter<10):
            print(video_string + " contains "+keyword+" with probability "+ str(inferences[v][class_id]))
            print_counter +=1
        video_strings.append(video_string)
        if inferences[v][class_id]>max_prob:
            max_prob = inferences[v][class_id]
            max_video = current_video
            max_time = start_time
    if(not video_strings):
        print("No videos found for the search query: "+keyword )
    else:
        strongest_video_embed = '<iframe width="560" height="315" src="https://www.youtube.com/embed/'+max_video+'?autoplay=1&start='+str(max_time)+';autoplay=1 frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>'
    return strongest_video_embed,video_strings

In [15]:
search_query = 'Rain'
embed = search_audio(search_query)[0]
if(embed):
    print("\n Playing video with the strongest probability of containing "+search_query+" ..")
HTML(embed)

Retrieving videos containing Rain with the highest probabilities...
...Done!
Printing a few strong results.. 
https://www.youtube.com/watch?v=-3-4qmWSJXU&t=34    30
Name: start_time, dtype: int64 contains Rain with probability 0.9672336
https://www.youtube.com/watch?v=0BauNGmZtTU&t=477    250
Name: start_time, dtype: int64 contains Rain with probability 0.99831176
https://www.youtube.com/watch?v=10rH37YYvKU&t=806    30
Name: start_time, dtype: int64 contains Rain with probability 0.9436661
https://www.youtube.com/watch?v=1B-RuZgSqf8&t=862    30
Name: start_time, dtype: int64 contains Rain with probability 0.90838933
https://www.youtube.com/watch?v=1De-OI8YG-M&t=877    7
Name: start_time, dtype: int64 contains Rain with probability 0.94205034

 Playing video with the strongest probability of containing Rain ..
